# Reglas de Particionamiento

## Descripcion de la base de datos

Dentro del campo de la ciencia de datos, nos interesa especialmente realizar análisis exploratorios sobre diversos tipos de datos. El objetivo es generar predicciones fundamentadas en el comportamiento pasado, lo que permite comprender mejor el comportamiento futuro.

Por esta razón, busqué una base de datos con información histórica y diversas categorías, que fuera ideal para realizar análisis exploratorios por categoría, comparaciones entre ellas y que contuviera el material necesario para generar predicciones futuras.

La base de datos Flight Status Predict cumplió con estas características. Además, aborda un tema que me resulta interesante: determinar qué factores podrían afectar las cancelaciones, retrasos, y las mejores aerolíneas para viajar en mis próximas vacaciones.

## Reglas de particionamiento Seleccionadas

- Ocurrencia entre nombre comercial y nombre operativo de aerolineas
- Ocurrencia entre Dia de la Semana y Mes
- Ocurrencia entre Nombre Comercial de la Aerolinea y `DistanceGroup`
- Ocurrencia vuelos entre la Aerolinea y el mes
- Ocurrencia de vuelos entre la Aerolinea y el dia de la semana
- Ocurrencia entre Aerolinea y la ciudad origen del vuelo
- Ocurrencia entre la Aerolinea y la ciudad destino en el vuelo
- Ocurrencia entre ciudad origen y destino

Nota: La variable `DistanceGroup` es un numero entero que representa una agrupacion de cada 500 millas recorridas por el vuelo del avion, es decir si `DistanceGroup = 2` el recorrido seria no mas de 1000 millas

In [1]:
import os
import time
import operator
import functools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, DateType, IntegerType, StringType, DoubleType, BooleanType

In [2]:
spark = SparkSession.builder \
                .appName("flights") \
                .config("spark.executor.memory", "10G") \
                .config("spark.driver.memory", "2G") \
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 21:24:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
schema = StructType(
    [
        StructField("FlightDate", DateType(), True),
        StructField("Airline", StringType(), True),
        StructField("Origin", StringType(), True),
        StructField("Dest", StringType(), True),
        StructField("Cancelled", BooleanType(), True),
        StructField("Diverted", BooleanType(), True),
        StructField("CRSDepTime", IntegerType(), True),
        StructField("DepTime", DoubleType(), True),
        StructField("DepDelayMinutes", DoubleType(), True),
        StructField("DepDelay", DoubleType(), True),
        StructField("ArrTime", DoubleType(), True),
        StructField("ArrDelayMinutes", DoubleType(), True),
        StructField("AirTime", DoubleType(), True),
        StructField("CRSElapsedTime", DoubleType(), True),
        StructField("ActualElapsedTime", DoubleType(), True),
        StructField("Distance", DoubleType(), True),
        StructField("Year", IntegerType(), True),
        StructField("Quarter", IntegerType(), True),
        StructField("Month", IntegerType(), True),
        StructField("DayofMonth", IntegerType(), True),
        StructField("DayOfWeek", IntegerType(), True),
        StructField("Marketing_Airline_Network", StringType(), True),
        StructField("Operated_or_Branded_Code_Share_Partners", StringType(), True),
        StructField("DOT_ID_Marketing_Airline", StringType(), True),
        StructField("IATA_Code_Marketing_Airline", StringType(), True),
        StructField("Flight_Number_Marketing_Airline", StringType(), True),
        StructField("Operating_Airline", StringType(), True),
        StructField("DOT_ID_Operating_Airline", StringType(), True),
        StructField("IATA_Code_Operating_Airline", StringType(), True),
        StructField("Tail_Number", StringType(), True),
        StructField("Flight_Number_Operating_Airline", StringType(), True),
        StructField("OriginAirportID", StringType(), True),
        StructField("OriginAirportSeqID", StringType(), True),
        StructField("OriginCityMarketID", StringType(), True),
        StructField("OriginCityName", StringType(), True),
        StructField("OriginState", StringType(), True),
        StructField("OriginStateFips", StringType(), True),
        StructField("OriginStateName", StringType(), True),
        StructField("OriginWac", StringType(), True),
        StructField("DestAirportID", StringType(), True),
        StructField("DestAirportSeqID", StringType(), True),
        StructField("DestCityMarketID", StringType(), True),
        StructField("DestCityName", StringType(), True),
        StructField("DestState", StringType(), True),
        StructField("DestStateFips", StringType(), True),
        StructField("DestStateName", StringType(), True),
        StructField("DestWac", StringType(), True),
        StructField("DepDel15", StringType(), True),
        StructField("DepartureDelayGroups", StringType(), True),
        StructField("DepTimeBlk", StringType(), True),
        StructField("TaxiOut", DoubleType(), True),
        StructField("WheelsOff", DoubleType(), True),
        StructField("WheelsOn", DoubleType(), True),
        StructField("TaxiIn", DoubleType(), True),
        StructField("CRSArrTime", IntegerType(), True),
        StructField("ArrDelay", DoubleType(), True),
        StructField("ArrDel15", StringType(), True),
        StructField("ArrivalDelayGroups", StringType(), True),
        StructField("ArrTimeBlk", StringType(), True),
        StructField("DistanceGroup", StringType(), True),
        StructField("DivAirportLandings", StringType(), True),
    ]
)

In [4]:
filepath = "./data/flights"
filenames = [ f"Combined_Flights_{y}.csv" for y in range(2018, 2023) ]

In [5]:
partition_rules = [('Marketing_Airline_Network', 'Operating_Airline'), ('DayOfWeek', 'Month'), ('Marketing_Airline_Network', 'DistanceGroup'), ('Marketing_Airline_Network', 'Month'),('Marketing_Airline_Network', 'DayOfWeek'), ('Marketing_Airline_Network', 'OriginCityName'), ('Marketing_Airline_Network', 'DestCityName'), ('OriginCityName', 'DestCityName')]
partition_rules_filenames = [ f"Combined_Flights_2018_Rule_{rule[0]}_{rule[1]}" for rule in partition_rules ]

# Muestreo basado en Reglas

El siguiente codigo filtra la base de datos de vuelos para obtener una muestra de acuerdo a las 3 reglas de mayor ocurrencia segun el paso anterior.

 - Se imprime la cantidad de filas resultantes para cada base de datos

In [ ]:
df = spark.read.schema(schema).csv(f"{filepath}/{filenames[0]}", header=True)

for filename, names in zip(partition_rules_filenames, partition_rules):
    rule = spark.read.parquet(f"{filepath}/{filename}")
    l = 3
    
    # rule.columns[:-1] son las dos columnas (p. ej. ['Marketing_Airline_Network','Operating_Airline']) usadas para la unión.
    # El inner join devuelve solo las filas de df cuyas tuplas en esas dos columnas coinciden con alguna de las 2 seleccionadas.
    # Luego recuperamos todas las columnas originales con .select(*df.columns)
    # Asumimos que el parquet de cada regla está ordenado por frecuencia descendente, de modo que limit(3) recoge las 3 combinaciones (tuplas de valores) que más se repiten según esa regla.
    database = df.join(rule.limit(l), rule.columns[:-1], 'inner').select(*df.columns)

    database.repartition(1).write.mode('overwrite').format('parquet').save(f"./data/flights/{filenames[0][:-4]}_Final_DB_{names[0]}-{names[1]}")
    
    # Impresion de la cantidad de filas resultantes en cada particion
    print(database.count())

time.sleep(1)

25/05/04 21:24:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1352552


297829


1109993


388495


675224


422343


422301


25741


In [7]:
spark.stop()